In [10]:
# Dependencies and Setup
import pandas as pd

# Files to Load
csv1 = "Resource/Coinbase_BCHUSD_d.csv"
csv2 = "Resource/Coinbase_BTCUSD_d.csv"
csv3 = "Resource/Coinbase_EOSUSD_d.csv"
csv4 = "Resource/Coinbase_ETCUSD_d.csv"
csv5 = "Resource/Coinbase_ETHUSD_d.csv"
csv6 = "Resource/Coinbase_LINKUSD_d.csv"
csv7 = "Resource/Coinbase_LTCUSD_d.csv"
csv8 = "Resource/Coinbase_REPUSD_d.csv"
csv9 = "Resource/Coinbase_XLMUSD_d.csv"
csv10 = "Resource/Coinbase_XRPUSD_d.csv"
csv11 = "Resource/COMP HistoricalPrices.csv"
csv12 = "Resource/DJIA HistoricalPrices.csv"
csv13 = "Resource/RUT 2000 HistoricalPrices.csv"
csv14 = "Resource/SPX 500 HistoricalPrices.csv"

csv_list = [csv1, csv2, csv3, csv4, csv5, csv6, csv7, csv8, csv9, csv10, csv11, csv12, csv13, csv14]

In [26]:
names = ['bch', 'btc', 'eos', 'etc', 'eth', 'lnk', 'ltc', 'rep', 'xlm', 'xrp', 'NAS', 'DOW', 'RUT', 'SPX']
data = {}  # dictionary that will hold them 

for index, csv in enumerate(csv_list):  # loop over files

    exec(f"{names[index]} = pd.read_csv(csv)")

,Date,Open,High,Low,Close
0,11/30/2020,29854.51,29854.51,29463.64,29638.64
1,11/27/2020,29911.33,30015.13,29819.98,29910.37
2,11/25/2020,30058.87,30058.87,29806.25,29872.47
3,11/24/2020,29746.11,30116.51,29746.11,30046.24
4,11/23/2020,29332.82,29667.75,29332.82,29591.27


In [27]:
bch.head()

,unix,date,symbol,open,high,low,close,Volume BCH,Volume USD
0,1599609600,9/9/2020,BCH/USD,222.28,224.10,218.53,222.64,840773.59,3806.41
1,1599523200,9/8/2020,BCH/USD,228.78,230.47,215.90,222.28,5037488.62,22526.33
2,1599436800,9/7/2020,BCH/USD,227.69,229.88,215.78,228.78,6226370.73,27795.53
3,1599350400,9/6/2020,BCH/USD,225.74,233.26,219.20,227.69,5232916.02,23027.00
4,1599264000,9/5/2020,BCH/USD,230.93,234.96,214.50,225.74,13992453.13,62468.00


In [25]:
data.keys()

dict_keys(['Resource/Coinbase_BCHUSD_d.csv', 'Resource/Coinbase_BTCUSD_d.csv', 'Resource/Coinbase_EOSUSD_d.csv', 'Resource/Coinbase_ETCUSD_d.csv', 'Resource/Coinbase_ETHUSD_d.csv', 'Resource/Coinbase_LINKUSD_d.csv', 'Resource/Coinbase_LTCUSD_d.csv', 'Resource/Coinbase_REPUSD_d.csv', 'Resource/Coinbase_XLMUSD_d.csv', 'Resource/Coinbase_XRPUSD_d.csv', 'Resource/COMP HistoricalPrices.csv', 'Resource/DJIA HistoricalPrices.csv', 'Resource/RUT 2000 HistoricalPrices.csv', 'Resource/SPX 500 HistoricalPrices.csv'])

In [11]:

data = pd.read_csv(csv5)
data.columns = [x.lower() for x in data.columns]
data

,unix timestamp,date,symbol,open,high,low,close,volume eth,volume usd
0,1605830400,11/20/2020,ETHUSD,471.92,488.47,471.07,484.88,61257.21,2.955847e+07
1,1605744000,11/19/2020,ETHUSD,478.96,480.99,465.18,471.92,131594.48,6.251464e+07
2,1605657600,11/18/2020,ETHUSD,482.68,495.68,456.50,478.96,296491.88,1.417250e+08
3,1605571200,11/17/2020,ETHUSD,460.85,485.00,460.05,482.68,196036.24,9.308297e+07
4,1605484800,11/16/2020,ETHUSD,448.58,466.21,445.91,460.85,112980.16,5.175862e+07
...,...,...,...,...,...,...,...,...,...
1622,1464652800,5/31/2016,ETHUSD,12.41,14.24,12.37,14.00,5654.91,7.699475e+04
1623,1464566400,5/30/2016,ETHUSD,12.34,12.88,12.00,12.41,4110.59,5.165595e+04
1624,1464480000,5/29/2016,ETHUSD,11.93,12.49,11.65,12.34,3481.30,4.222837e+04
1625,1464393600,5/28/2016,ETHUSD,11.25,12.44,10.70,11.93,15810.34,1.808220e+05


In [12]:
min_dates = []
max_dates = []

for csv in csv_list:
    data = pd.read_csv(csv)
    data.columns = [x.lower() for x in data.columns]
    min_date = data['date'].min()
    max_date = data['date'].max()
    min_dates.append(min_date)
    max_dates.append(max_date)

In [13]:
data_dict = {'File': csv_list, 'Earliest Date': min_dates, 'Last Date': max_dates}
df = pd.DataFrame(data_dict)
df

,File,Earliest Date,Last Date
0,Resource/Coinbase_BCHUSD_d.csv,1/1/2018,9/9/2020
1,Resource/Coinbase_BTCUSD_d.csv,1/1/2015,9/9/2020
2,Resource/Coinbase_EOSUSD_d.csv,1/1/2020,9/9/2020
3,Resource/Coinbase_ETCUSD_d.csv,1/1/2019,9/9/2020
4,Resource/Coinbase_ETHUSD_d.csv,1/1/2017,9/9/2020
5,Resource/Coinbase_LINKUSD_d.csv,1/1/2020,9/9/2020
6,Resource/Coinbase_LTCUSD_d.csv,1/1/2017,9/9/2020
7,Resource/Coinbase_REPUSD_d.csv,1/1/2020,9/9/2020
8,Resource/Coinbase_XLMUSD_d.csv,1/1/2020,9/9/2020
9,Resource/Coinbase_XRPUSD_d.csv,1/1/2020,9/9/2020


In [17]:
df['Earliest Date'] = pd.to_datetime(df['Earliest Date'])

In [18]:
df.sort_values(by='Earliest Date')

,File,Earliest Date,Last Date
10,Resource/COMP HistoricalPrices.csv,2014-01-10,9/9/2020
11,Resource/DJIA HistoricalPrices.csv,2014-01-10,9/9/2020
12,Resource/RUT 2000 HistoricalPrices.csv,2014-01-10,9/9/2020
13,Resource/SPX 500 HistoricalPrices.csv,2014-01-10,9/9/2020
1,Resource/Coinbase_BTCUSD_d.csv,2015-01-01,9/9/2020
4,Resource/Coinbase_ETHUSD_d.csv,2017-01-01,9/9/2020
6,Resource/Coinbase_LTCUSD_d.csv,2017-01-01,9/9/2020
0,Resource/Coinbase_BCHUSD_d.csv,2018-01-01,9/9/2020
3,Resource/Coinbase_ETCUSD_d.csv,2019-01-01,9/9/2020
2,Resource/Coinbase_EOSUSD_d.csv,2020-01-01,9/9/2020


#Creating Data Table 

AttributeError: 'str' object has no attribute 'head'